In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

from sqlalchemy import and_
from sqlalchemy.sql import select

from exphewas.db.models import *
from exphewas.db.engine import Session
import exphewas.db.tree as tree
from exphewas.backend.r_bindings import R as R_

In [2]:
%matplotlib inline

In [3]:
R = R_()

In [4]:
chembl = pd.read_csv("../data/chembl/chembl.csv.gz")
chembl.head()

,who_name,level1,level2,level3,level4,level5,level1_description,level2_description,level3_description,level4_description,...,disease_efficacy,mechanism_comment,selectivity_comment,binding_site_comment,component_id,component_type,accession,description,organism,db_source
0,oprelvekin,L,L03,L03A,L03AC,L03AC02,ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS,IMMUNOSTIMULANTS,IMMUNOSTIMULANTS,Interleukins,...,1,complex with beta but only associcates on liga...,NaN,NaN,390,PROTEIN,Q14626,Interleukin-11 receptor subunit alpha,Homo sapiens,SWISS-PROT
1,levosimendan,C,C01,C01C,C01CX,C01CX08,CARDIOVASCULAR SYSTEM,CARDIAC THERAPY,CARDIAC STIMULANTS EXCL. CARDIAC GLYCOSIDES,Other cardiac stimulants,...,1,It is a Ca2+ sensitizer. It acts through direc...,NaN,NaN,2231,PROTEIN,Q13370,"cGMP-inhibited 3',5'-cyclic phosphodiesterase B",Homo sapiens,SWISS-PROT
2,prilocaine,N,N01,N01B,N01BB,N01BB04,NERVOUS SYSTEM,ANESTHETICS,"ANESTHETICS, LOCAL",Amides,...,1,NaN,NaN,NaN,2613,PROTEIN,Q15858,Sodium channel protein type 9 subunit alpha,Homo sapiens,SWISS-PROT
3,nandrolone,A,A14,A14A,A14AB,A14AB01,ALIMENTARY TRACT AND METABOLISM,ANABOLIC AGENTS FOR SYSTEMIC USE,ANABOLIC STEROIDS,Estren derivatives,...,1,NaN,NaN,NaN,187,PROTEIN,P10275,Androgen receptor,Homo sapiens,SWISS-PROT
4,trastuzumab emtansine,L,L01,L01X,L01XC,L01XC14,ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS,ANTINEOPLASTIC AGENTS,OTHER ANTINEOPLASTIC AGENTS,Monoclonal antibodies,...,1,inhibits polymerisation,NaN,Binds to beta tubulin,1976,PROTEIN,Q71U36,Tubulin alpha-1A chain,Homo sapiens,SWISS-PROT


In [6]:
# Corresponding Ensembl IDs of genes encoding drug targets for the given ATC code.
con = Session()

# -1 is the id for the Uniprot to Ensembl mapping
query = select([XRefs.external_id, XRefs.ensembl_id])\
  .where(XRefs.external_db_id == -1)

uniprot_to_ensembl = {k: v for k, v in con.execute(query)}

In [7]:
# As a test we will check lipid drugs with the self-reported disease outcomes list and visualize.
atc_tree = tree.tree_from_hierarchy_id("ATC")

In [8]:
# Get results for self-reported diabetes.
outcome_results = Session()\
    .query(BinaryVariableResult.gene, BinaryVariableResult.p)\
    .filter_by(outcome_id="1245")\
    .filter_by(variance_pct=95)\
    .order_by(BinaryVariableResult.p)\
    .all()

In [10]:
ranks = pd.DataFrame(outcome_results, columns=["gene", "p"])
ranks = ranks.set_index("gene", verify_integrity=True)
ranks.head()

,p
gene,
ENSG00000204296,0.0
ENSG00000237541,0.0
ENSG00000204305,0.0
ENSG00000204538,0.0
ENSG00000204387,0.0


In [11]:
atc_to_target = []
for i, atc in atc_tree.iter_depth_first():
    chembl_atc_codes = getattr(chembl, f"level{i}")
        
    # Get targets.
    targets = chembl.loc[chembl_atc_codes == atc.code, "accession"].drop_duplicates()
    
    # We skip classes with few known targets because it's not
    # useful for enrichment.
    if len(targets) <= 1:
        # No known targets for this class.
        continue
    
    # Convert targets to Ensembl.
    for t in targets:
        ensg = uniprot_to_ensembl.get(t)
        
        if ensg is None:
            continue
            
        atc_to_target.append((atc.code, ensg))

atc_to_target = pd.DataFrame(atc_to_target, columns=["atc", "target"])

In [28]:
atc_to_target["x"] = 1
m = pd.pivot_table(atc_to_target, columns=["atc"], index=["target"], values="x", fill_value=0)
included_atc = m.columns
df = ranks.join(m)
df = df.fillna(0)
df["q"] = R.qvalue(df.p.values)

In [29]:
df["sig"] = (df["q"] <= 0.01).astype(int)

In [31]:
results = []
for atc in included_atc:
    # Build a contingency table
    #print(df[["sig", atc, "p"]].groupby(["sig", atc]).count())
    contingency = pd.crosstab(df.sig, df[atc]).values
    if contingency.shape != (2, 2):
        print(f"Weird contingency for {atc}")
        continue
    
    or_, p = scipy.stats.fisher_exact(contingency)
    results.append((atc, or_, p))

results = pd.DataFrame(results, columns=["ATC", "OR", "p"]).sort_values("p")

Weird contingency for N06AF


In [34]:
results

,ATC,OR,p
383,V04CA,inf,0.000865
37,A10BB,inf,0.000865
39,A10BX,66.182149,0.002544
36,A10B,12.427007,0.003508
35,A10,11.045620,0.004576
...,...,...,...
190,L01BC,0.000000,1.000000
57,B06A,0.000000,1.000000
188,L01BA,0.000000,1.000000
55,B02BX,0.000000,1.000000
